In [1]:
# Import Libraries and Dependencies
import constrained_optimization
from constrained_optimization.base import*
from constrained_optimization.generate import*

In [2]:
dimX = 10
dimY = 10
dimU = 10

import time
start_time = time.time()
p, q, r, A, b = randQCQP(dimX,dimY,True,True, dimU)
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.0029954910278320312 seconds ---


In [ ]:
start_time = time.time()
p, q, r, A, b = randQCQPmat(dimX,dimY,True,True, dimU)
print("--- %s seconds ---" % (time.time() - start_time))

In [3]:
M_ = 3
N_ = 2
convex_ = False

x_rand = np.random.rand(N_)
q_temp = np.random.rand(M_+1, N_)
p_temp = np.random.rand(M_+1, N_, N_)
if convex_:
    p_temp = np.matmul(p_temp, np.transpose(p_temp, (0,2,1)))
else :
    p_temp = np.maximum(p_temp, np.transpose(p_temp, (0,2,1)))
r_temp = -0.5*np.transpose(x_rand)@p_temp@x_rand - q_temp@x_rand

In [ ]:

start_time = time.time()
fx, hx, gradf, gradh = toQCQPmat(p, q, r)
print("--- %s seconds ---" % (time.time() - start_time))
hxs = []
gradhs = []
for i in range(len(hx)):
    temp = lambda x: -1*hx[i](x)
    hxs.append(temp)
    temp = lambda x: -1*gradh[i](x)
    gradhs.append(temp)

In [3]:
fx, hx, gradf, gradh = toQCQP(p, q, r)

In [5]:
prob = co_problem(fx, dimX, hx, None, gradf, gradh, A, b)

x = np.ones(30)
F_co, J_co = prob.get_parameters(False)
F_co(x)

array([ 2.42324760e+03,  2.43984058e+03,  2.43027020e+03,  2.43414731e+03,
        2.41971903e+03,  2.43176728e+03,  2.43011419e+03,  2.42665037e+03,
        2.43047491e+03,  2.43240428e+03, -9.23462760e+01, -9.23462760e+01,
       -9.23462760e+01, -9.23462760e+01, -9.23462760e+01, -9.23462760e+01,
       -9.23462760e+01, -9.23462760e+01, -9.23462760e+01, -9.23462760e+01,
       -2.43490312e+00, -2.19438546e+00, -2.21848317e+00, -2.29098450e+00,
       -3.50613026e+00, -2.24440923e+00, -2.38752034e+00, -2.13819043e+00,
       -2.87736810e+00, -2.08312134e+00])

## Adaptive Graal to solve VI

In [ ]:
def adaptive_graal(J, F, prox_g, x1, numb_iter=100, phi=1.5, output=False): 
    begin = perf_counter()
    x, x_ = x1.copy(), x1.copy()
    x0 = x + np.random.randn(x.shape[0]) * 1e-9
    Fx = F(x)
    la = phi / 2 * LA.norm(x - x0) / LA.norm(Fx - F(x0))
    rho = 1. / phi + 1. / phi**2
    values = [J(x)]
    time_list = [perf_counter() - begin]
    th = 1

    for i in range(numb_iter):
        x1 = prox_g(x_ - la * Fx, la)
        Fx1 = F(x1)

        n1 = LA.norm(x1 - x)**2
        n2 = LA.norm(Fx1 - Fx)**2
        n1_div_n2 = n1/n2 if n2 != 0 else la*10

        la1 = min(rho * la, 0.25 * phi * th / la * n1_div_n2)
        x_ = ((phi - 1) * x1 + x_) / phi
        if output:
            print (i, la)
        th = phi * la1 / la
        x, la, Fx = x1, la1, Fx1
        values.append(J(x))
        time_list.append(perf_counter() - begin)
    end = perf_counter()

    print("CPU time for aGRAAL:", end - begin)
    return values, x, x_, time_list
def adaptive_graal_terminate(J, F, prox_g, x1, numb_iter=100, phi=1.5, tol=1e-6, output=False):
    begin = perf_counter()
    x, x_ = x1.copy(), x1.copy()
    x0 = x + np.random.randn(x.shape[0]) * 1e-9
    Fx = F(x)
    la = phi / 2 * LA.norm(x - x0) / LA.norm(Fx - F(x0))
    rho = 1. / phi + 1. / phi**2
    values = [J(x)]
    time_list = [perf_counter() - begin]
    th = 1

    i = 1
    while i <= numb_iter and values[-1] > tol:
        i += 1
        x1 = prox_g(x_ - la * Fx, la)
        Fx1 = F(x1)

        n1 = LA.norm(x1 - x)**2
        n2 = LA.norm(Fx1 - Fx)**2
        n1_div_n2 = np.exp(np.log(n1) - np.log(n2))
        la1 = min(rho * la, 0.25 * phi * th / la * n1_div_n2, 1e6)
        x_ = ((phi - 1) * x1 + x_) / phi
        if output:
            print (i, la)

        th = phi * la1 / la
        x, la, Fx = x1, la1, Fx1
        values.append(J(x))
        time_list.append(perf_counter() - begin)

    end = perf_counter()

    print("CPU time for aGRAAL:", end - begin)
    return values, x, i


## Solver of VI

In [ ]:
prox_g = lambda q, eps: np.fmax(q,0)
# starting point
N = 1000
q0 = np.ones(dimX + dimY + dimU)
print(F_co(q0).shape)

In [ ]:
ans1 = adaptive_graal(J_co, F_co, prox_g, q0, numb_iter=N, phi=1.5, output=False)

In [ ]:
plt.plot(ans1[0], '--b', label = 'FBF')

plt.yscale('log')

plt.xlabel(u' iterations, $k$')
plt.ylabel(u'residual')

plt.legend()
#plt.savefig('figures/nash.pdf', bbox_inches='tight')
plt.show()
print("Answer is", ans1[0][-1])
print("Answer is", ans1[2])
print("Answer is", fx(ans1[2][0:dimX]))

## Solving the QCQP

In [ ]:
from scipy.optimize import minimize
cons = []
for i in range(len(hx)):
    temp = ({'type': 'ineq', 'fun': lambda x: hx[i](x) })
    cons.append(temp)
bnds = []
for i in range(dimX):
    temp = ((0, None))
    bnds.append(temp)
x = np.ones(dimX)
res = minimize(fx, x, method='SLSQP', bounds=bnds, constraints=cons)

In [ ]:
print(fx(res['x']))
print(res['x'])
#print("Answer is:", res)